ModuleNotFoundError: No module named 'bson'

In [136]:
# Import the Twython class
import tweepy
import csv
import time
import json
import datetime
import pandas as pd
import os
from json import dumps

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

In [157]:
#Mysql
import pymysql
import json
from functools import partial
import csv

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)
            
def return_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        for result in cursor:
            return result #retorna logo o primeiro
connection = pymysql.connect(
    host='localhost',
    user='megadados',
    password='linux123',
    database='redes')

db = partial(run_db_query, connection)


In [158]:
#Função do DB para retornar as tags dos jogadores
def return_players_tag(connection,id_jogador):    

    with connection.cursor() as cursor:

        cursor.execute('SELECT tag FROM jogador_tag WHERE id_jogador = %s', (id_jogador) ) ## irá retornar a tag do jogador no twitter

        res = cursor.fetchall()

        tags = tuple(x[0] for x in res)

        return tags

In [159]:
db('SELECT * FROM jogador_tag')


Executando query:
(1, 'alaatweets')
(3, 'kaj33')
(6, 'ShareefAbdur_R')
(10, 'alexabrines')
(15, 'QuincyAcy')
(19, '10jadams')
(20, 'jordanadams1231')
(22, 'RealStevenAdams')
(24, 'Bam1of1')
(27, 'Adrien4')
(29, 'moeager')
(32, 'danielrainge')
(33, 'AjincaAlexis42')
(40, 'furkanaldemir19')
(41, 'colea45')
(42, 'aldridge_12')
(44, 'CAlexander')
(52, 'Iam_RawleAlkins')
(54, 'GraysonJAllen')
(55, '_bigjayy_')
(60, 'THEleekAllen30')
(62, 'greenRAYn20')
(63, 'aa000G9')
(67, 'FreeMoAlmond')
(71, 'JohnAmaechi')
(73, 'farouq1')
(74, 'LouAmundson17')
(78, 'ASqared06')
(88, 'JusAnderson1')
(89, 'chibbs_1')
(91, 'KyleAnderson5')
(103, 'IkePostrs_')
(106, 'Giannis_An34')
(107, 'Kostas_ante13')
(109, 'carmeloanthony')
(110, 'GregAnthony50')
(113, 'OAnunoby')
(118, 'RobertArchibald')
(119, 'RyArch15')
(121, 'agentzeroshow')
(122, 'TrevorAriza')
(125, 'bjarmstrong')
(136, 'DarrellArthur00')
(138, 'melartis1')
(139, 'AsikOmer')
(157, 'Gustavo_Ayon15')
(159, 'DeandreAyton')
(160, 'KAzubuike7')
(162, 'Re

In [160]:
tags = return_players_tag(connection,22)
print(tags[0])
user_tweet = tags[0]

RealStevenAdams


In [161]:

class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    with open("twitter_credentials.json", "r") as file:
        creds = json.load(file)
    
    
    def __init__(self, keys_dict=creds, api=api,
                 result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(
            creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
        auth.set_access_token(
            creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit

        
    def mine_user_tweets(self, user=user_tweet, #BECAUSE WHO ELSE!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                    mined['text'] = re.sub(r"http\S+", "", mined[''])
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 
#def save_to_json(tweet):
    #with open("saved_tweets.json", "wt") as file:
        #json.dump(tweet, file, indent = 4)

In [162]:
miner=TweetMiner(result_limit = 200)

In [163]:
mined_tweets = miner.mine_user_tweets(user=user_tweet,max_pages=17)

mined_tweets_df= pd.DataFrame(mined_tweets)

In [154]:

handle_list= ['list of handles you want the timelines of']

twitter_dict={}
counter=0

for name in handle_list:
    try:
      twitter_dict[name]=[]
      twitter_dict[name].append(miner.mine_user_tweets(user=name, max_pages=17))
      counter = counter +1
      if counter%40==0:
        time.sleep(900) #15 minute sleep time
    #if name invalid print name and remove key
    except:
      print(name, 'is invalid or locked')
      twitter_dict.pop(name)
    
all_tweets=pd.concat([pd.DataFrame(twitter_dict[i][0]) for i in twitter_dict])

In [155]:
pd.DataFrame(mined_tweets)


,tweet_id,name,screen_name,retweet_count,text,mined_at,created_at,favourite_count,hashtags,status_count,location,source_device,retweet_text,quote_text,quote_screen_name
0,1188110271497281536,Steven Adams,RealStevenAdams,1578,RT @ufc: Maia does it!\n\n@DemianMaia submits ...,2019-11-01 17:26:29.946607,2019-10-26 15:08:57,0,"[{'text': 'UFCSingapore', 'indices': [70, 83]}]",1495,None,Twitter for Android,Maia does it!\n\n@DemianMaia submits Askren in...,None,None
1,1186098602155872262,Steven Adams,RealStevenAdams,66,RT @alexabrines: Seguimos!!! Victoria con unas...,2019-11-01 17:26:29.946644,2019-10-21 01:55:18,0,"[{'text': 'SúmateAlRosa', 'indices': [83, 96]}...",1495,None,Twitter for Android,Seguimos!!! Victoria con unas zapatillas con m...,None,None
2,1183495023586959360,Steven Adams,RealStevenAdams,102,🙌🏽🙌🏽🙌🏽 well done lads https://t.co/vg9QMwDsAR,2019-11-01 17:26:29.946656,2019-10-13 21:29:36,1229,[],1495,None,Twitter for Android,None,None,None
3,1181568802565345287,Steven Adams,RealStevenAdams,7,RT @DanielPryorr: Want to change the world doi...,2019-11-01 17:26:29.946670,2019-10-08 13:55:29,0,[],1495,None,Twitter for Android,Want to change the world doing a job you love?...,None,None
4,1180704421585592321,Steven Adams,RealStevenAdams,129,Style bender!,2019-11-01 17:26:29.946680,2019-10-06 04:40:45,1612,[],1495,None,Twitter for Android,None,None,None
5,1170450112075784192,Steven Adams,RealStevenAdams,0,@ChrisReive 🙌🏽,2019-11-01 17:26:29.946690,2019-09-07 21:33:47,10,[],1495,None,Twitter for Android,None,None,None
6,1170438442649608200,Steven Adams,RealStevenAdams,46,🦅🦅🦅,2019-11-01 17:26:29.946699,2019-09-07 20:47:25,778,[],1495,None,Twitter for Android,None,None,None
7,1170150275450994689,Steven Adams,RealStevenAdams,1,@SingSing Lemon honey ginger W splash of whisk...,2019-11-01 17:26:29.946709,2019-09-07 01:42:20,84,[],1495,None,Twitter for Android,None,None,None
8,1166978167157841921,Steven Adams,RealStevenAdams,5,RT @chrisdrmcintyre: Very excited to bring Pol...,2019-11-01 17:26:29.946719,2019-08-29 07:37:31,0,[],1495,None,Twitter for Android,Very excited to bring Policy Local to life in ...,None,None
9,1165848174474477569,Steven Adams,RealStevenAdams,12,RT @DakariJohnson: Forever grateful and apprec...,2019-11-01 17:26:29.946728,2019-08-26 04:47:19,0,[],1495,None,Twitter for Android,Forever grateful and appreciative for this opp...,None,None


In [156]:
# Salvar json tweets

try:
    os.remove("saved_tweets.json")
except:
    pass


mined_tweets_df.to_json(r'saved_tweets.json')


